<center>
<hr>
<h1>Artificial stock market of synthetic index</h1>
<h3>Laurea Magistrale in Fisica Dei Sistemi Complessi</h3> 
<h3>A.A. 2018/19</h3>
<h3>Gianmarco Sabbatini, Matteo Mazza, Lorenzo Costantini</h3>
<hr>
</center>

In [1]:
import random as rd
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
from xcs import XCSAlgorithm
from xcs.scenarios import ScenarioObserver
from xcs.scenarios import Scenario
from xcs.bitstrings import BitString
import datetime

**CompleteCalendar** is the function which takes the rough data and transform them completing the calendar, adding the "missing" data.

In [2]:
def CompleteCalendar(dati):
    year=[]
    month=[]
    day=[]
    for i in range(len(dati)):
        year.append(int(dati.Date[i][0:4]))
        month.append(int(dati.Date[i][5:7]))
        day.append(int(dati.Date[i][8:10]))
    
    calendario=[]
    for i in range(len(dati)):
        calendario.append(datetime.datetime(year[i], month[i], day[i]).weekday())
     
    for i in range(len(dati)):
        vol.append(dati.Volume[i])
        
    for i in range(len(dati)):
        data.append(dati.Close[i])
    
    index=[]
    for i in range(0,len(dati)-1):
        if (calendario[i]==4 and calendario[i+1]!=0):
            index.append(i)
        elif (calendario[i]!=(calendario[i+1]-1) and calendario[i]!=4):
            index.append(i)
                    
    c=-1      
    for i in index:
        c+=1
        if (calendario[i+c]==4):
            calendario.insert(i+1+c,0)
        else:
            calendario.insert(i+1+c,calendario[i+c]+1)
    
    c=-1      
    for i in index:
        c+=1
        data.insert(i+1+c, data[i+c])
        vol.insert(i+1+c, 0)

**AGENTS**: 

In this section the structure of the agents is created.

**Arbitrajeur**: this agent is the one which keep aligned the artificial stock market with the real data. He act only in the stock market.

In [3]:
class Arbitrajeur:
    
    def __init__(self,id):
        self.id=id
        self.gain=0 #guadagno
        self.cash=0 
        self.stock=0
    
    
    def Action(self, data, price ):  
        if(data>price+0.018*price):
            decision = 'Buy'
        elif (data<price-0.018*price):
            decision ='Sell'
        else:
            decision='Pass'
        return decision
    
    def PriceBuy(self, priceS):
        return priceS+(float(priceS)*0.001)
   
    def PriceSell(self, priceB):
        return priceB-(float(priceB)*0.001)

**Random agent**: this agent is the one which bring the stocasticity in the markets. It acts both in the stock markets and in the index market.

In [4]:
class RandomAgent: 
    def __init__(self,id):
        self.id=id
        self.gain=0 
        self.cash=0 
        self.stock=0
    
    def Action(self):
        decision= rd.random()
        if(decision<=0.45):
            decision='Buy'
        elif(decision<0.9):
            decision='Sell'
        else:
            decision='Pass'
        return decision
    
    def Price(self, price):
        return float(price)+rd.gauss(0, price/100)

**Index Arbitrajeur**: this agent is the one which keep aligned the value of the index with the value of the underlying stocks. It acts only on the index market.

In [ ]:
class Arbitrajeur_index:
        
    def __init__(self,id):
        self.id=id
        self.gain=0 #guadagno
        self.cash=0 
        self.stock=0
    
    
    def Action(self, data, price ):  
        if(data>price+0.018*price):
            decision = 'Buy'
        elif (data<price-0.018*price):
            decision ='Sell'
        else:
            decision='Pass'
        return decision
    
    def PriceBuy(self, priceS):
        return priceS+(float(priceS)*0.001)
   
    def PriceSell(self, priceB):
        return priceB-(float(priceB)*0.001)

**Data loading**

In [ ]:
#fca=pd.read_csv("FCA.csv", sep=",")
apple=pd.read_csv("AAPL.csv",sep=",")
amazon=pd.read_csv("AMZN.csv",sep=",")
toyota=pd.read_csv("TM.csv",sep=",")
microsoft=pd.read_csv("MSFT.csv",sep=",")

**Data preparing**

In [ ]:
'''vol=[]
data=[]
CompleteCalendar(fca)
vol_fca=[]
data_fca=[]
vol_fca=vol
data_fca=data'''


'vol=[]\ndata=[]\nCompleteCalendar(fca)\nvol_fca=[]\ndata_fca=[]\nvol_fca=vol\ndata_fca=data'

In [ ]:
vol=[]
data=[]
CompleteCalendar(apple)
vol_apple=[]
data_apple=[]
vol_apple=vol
data_apple=data

In [ ]:
vol=[]
data=[]
CompleteCalendar(amazon)
vol_amazon=[]
data_amazon=[]
vol_amazon=vol
data_amazon=data

In [ ]:
vol=[]
data=[]
CompleteCalendar(toyota)
vol_toyota=[]
data_toyota=[]
vol_toyota=vol
data_toyota=data

In [ ]:
vol=[]
data=[]
CompleteCalendar(microsoft)
vol_microsoft=[]
data_microsoft=[]
vol_microsoft=vol
data_microsoft=data

**Markets**

In this section the two types of market's structure are created. They are similar in all their parts: they manage their book and the transaction on it. The difference is only one: the stock market have an attrbute that is *storic_data* which is the real time serie. This time serie is nonexistent for the index because is created by ourselves. The index market however keep trace of the real value of the underlying stock markets. 

**Stock Market**: this market represents the book of the stocks with all the function needed to manage it

In [ ]:
class Market:
    
    def __init__(self,storic_data,storic_volume):
        self.queueB=[]
        self.queueS=[]
        self.currentprice=storic_data[0]
        self.volume=storic_volume
        self.data=storic_data
        self.finalday=[]
        self.finalday.append(storic_data[0])
        
    def transaction(self,agents):
        if(len(self.queueB)!=0 and len(self.queueS)!=0):
            if(self.queueB[-1][0]>self.queueS[0][0]):
                for j in range(len(agents)):
                    if(self.queueB[-1][1]==agents[j].id):
                        agents[j].cash-=self.queueS[0][0]
                        agents[j].stock += 1
                    elif(self.queueS[0][1]==agents[j].id):
                        agents[j].cash+=self.queueS[0][0]
                        agents[j].stock -= 1
                self.currentprice = self.queueS[0][0]
                self.queueB.remove(self.queueB[-1])
                self.queueS.remove(self.queueS[0])
                return True
            else:
                return False
        
    def deletequeue(self):
        self.queueB.clear() #clear rimuove tutti gli elementi di una lista
        self.queueS.clear()
        
    def modifyqueueB(self, insertprize, identity): #identity è l'id dell'agente
        link=[]
        link.append(insertprize)
        link.append(identity)
        self.queueB.append(link)
        self.queueB.sort()
        
    def modifyqueueS(self, insertprize, identity):
        link=[]
        link.append(insertprize)
        link.append(identity)
        self.queueS.append(link)
        self.queueS.sort()

**Index Market**: this market represents the book of the index with all the function needed to manage it

In [ ]:
class Market_Index:
    
    def __init__(self,storic_data1, storic_data2,storic_data3, storic_data4) :
        self.queueB=[]
        self.queueS=[]      
        self.finalday=[]
        self.subprice=100
        self.currentprice=self.subprice
        self.constant=100/((storic_data1[0]+storic_data2[0]+storic_data3[0]+0.05*storic_data4[0])/3.05)
        self.finaldaysub=[]
        self.finaldaysub.append(100)
        self.finalday.append(100)
        
    def transaction(self,agents):
        if(len(self.queueB)!=0 and len(self.queueS)!=0):
            if(self.queueB[-1][0]>self.queueS[0][0]):
                for j in range(len(agents)):
                    if(self.queueB[-1][1]==agents[j].id):
                        agents[j].cash-=self.queueS[0][0]
                        agents[j].stock += 1
                    elif(self.queueS[0][1]==agents[j].id):
                        agents[j].cash+=self.queueS[0][0]
                        agents[j].stock -= 1
                self.currentprice = self.queueS[0][0]
                self.queueB.remove(self.queueB[-1])
                self.queueS.remove(self.queueS[0])
                return True
            else:
                return False
        
    def deletequeue(self):
        self.queueB.clear() #clear rimuove tutti gli elementi di una lista
        self.queueS.clear()
        
    def modifyqueueB(self, insertprize, identity): #identity è l'id dell'agente
        link=[]
        link.append(insertprize)
        link.append(identity)
        self.queueB.append(link)
        self.queueB.sort()
        
    def modifyqueueS(self, insertprize, identity):
        link=[]
        link.append(insertprize)
        link.append(identity)
        self.queueS.append(link)
        self.queueS.sort()
        
    def modify_currentsubprice(self):
        self.subprice=((apple_M.currentprice+amazon_M.currentprice+microsoft_M.currentprice+0.05*toyota_M.currentprice)/3.05)*self.constant

**Creating a list of markets** with 4 stock markets and one index market

In [ ]:
toyota_M = Market(data_toyota, vol_toyota)
apple_M = Market(data_apple, vol_apple)
amazon_M = Market(data_amazon, vol_amazon)
microsoft_M=Market(data_microsoft, vol_microsoft)
index_M=Market_Index(data_apple, data_amazon, data_microsoft, data_toyota)
list_M=[]
list_M.append(toyota_M)
list_M.append(apple_M)
list_M.append(amazon_M)
list_M.append(microsoft_M)
list_M.append(index_M)


**Creating a list of agents**

In [ ]:
agentlist=[]
N_random=1
N_arbitrajeur=1
N_index_arbitrajeur=1
for i in range(N_random):
    A=RandomAgent(i)
    agentlist.append(A)
for i in range(N_arbitrajeur):
    A=Arbitrajeur(N_random+i)
    agentlist.append(A)
for i in range(N_index_arbitrajeur):
    A=Arbitrajeur_index(N_random+N_arbitrajeur+i)
    agentlist.append(A)

**Scheduling of the action of the agents**

In [ ]:
a=0
b=0
for t in range(1, 900):
    for i in range (2000):
        if i<240:
            a=0.1
            b=rd.random()
        elif (i<1990):
            a=rd.random()
            b=rd.random()
        else:
            a=0.95
            b=rd.random()
            
            
        if (b>0.8):
            market=list_M[4]
            if a>0.7:
                agent=agentlist[2]
            else:
                agent=agentlist[0]

        elif (b>0.6):
            market=list_M[3]
            if (a>0.78):
                agent=agentlist[1]
            else:
                agent=agentlist[0]

        elif (b>0.4):
            market=list_M[2]
            if (a>0.78):
                agent=agentlist[1]
            else:
                agent=agentlist[0]

        elif (b>0.2):
            market=list_M[1]
            if (a>0.78):
                agent=agentlist[1]
            else:
                agent=agentlist[0]

        else:
            market=list_M[0]
            if (a>0.78):
                agent=agentlist[1]
            else:
                agent=agentlist[0]
        
        if(type(agent)==RandomAgent):
            action=agent.Action()
            if (action=='Buy'):
                market.modifyqueueB(agent.Price(market.finalday[t-1]),agent.id)
                if (market.transaction(agentlist)==True and type(market)==Market):
                    index_M.modify_currentsubprice()
            if (action=='Sell'):
                market.modifyqueueS(agent.Price(market.finalday[t-1]),agent.id)
                if (market.transaction(agentlist)==True and type(market)==Market):
                    index_M.modify_currentsubprice()


        elif(type(agent)==Arbitrajeur and type(market)!=Market_Index):
            action=agent.Action(market.data[t], market.currentprice)
            if (action=='Buy'):
                if (len(market.queueS)!=0):
                    market.modifyqueueB(agent.PriceBuy(market.queueS[0][0]),agent.id)
                else:
                    market.modifyqueueB(agent.PriceBuy(market.currentprice*1.01),agent.id)
                if (market.transaction(agentlist)==True and type(market)==Market):
                    index_M.modify_currentsubprice()
            if (action=='Sell'):
                if (len(market.queueB)!=0):
                    market.modifyqueueS(agent.PriceSell(market.queueB[-1][0]),agent.id)
                else:
                    market.modifyqueueB(agent.PriceBuy(market.currentprice*(1/1.01)),agent.id)
                if (market.transaction(agentlist)==True and type(market)==Market):
                    index_M.modify_currentsubprice()


        elif(type(agent)==Arbitrajeur_index and type(market)!=Market):
            action=agent.Action(market.subprice, market.currentprice)
            if (action=='Buy'):
                if (len(market.queueS)!=0):
                    market.modifyqueueB(agent.PriceBuy(market.queueS[0][0]),agent.id)
                else:
                    market.modifyqueueB(agent.PriceBuy(market.currentprice*1.01),agent.id)
                market.transaction(agentlist)
                
                for i in range (0,3):
                    if (len(list_M[i].queueB)!=0):
                        list_M[i].modifyqueueS(agent.PriceSell(list_M[i].queueB[-1][0]),agent.id)
                    else:
                        list_M[i].modifyqueueB(agent.PriceBuy(list_M[i].currentprice*(1/1.01)),agent.id)
                    if (list_M[i].transaction(agentlist)==True and type(list_M[i])==Market):
                        index_M.modify_currentsubprice()
                    
                    
            if (action=='Sell'):
                if (len(market.queueB)!=0):
                    market.modifyqueueS(agent.PriceSell(market.queueB[-1][0]),agent.id)
                else:
                    market.modifyqueueB(agent.PriceBuy(market.currentprice*(1/1.01)),agent.id)
                market.transaction(agentlist)
            
                for i in range (0,3):
                    if (len(list_M[i].queueS)!=0):
                        list_M[i].modifyqueueB(agent.PriceBuy(list_M[i].queueS[0][0]),agent.id)
                    else:
                        list_M[i].modifyqueueB(agent.PriceBuy(list_M[i].currentprice*1.01),agent.id)
                    market.transaction(agentlist)



    for market in list_M:
        market.finalday.append(market.currentprice)
        if type(market)==Market_Index:
            market.finaldaysub.append(market.subprice)
        market.deletequeue
                

In [ ]:
plt.plot(index_M.finalday)
plt.plot(index_M.finaldaysub)

In [ ]:
plt.plot(toyota_M.finalday)
plt.plot(data_toyota[0:900])

In [ ]:
plt.plot(amazon_M.finalday)
plt.plot(data_amazon[0:900])

In [ ]:
plt.plot(microsoft_M.finalday)
plt.plot(data_microsoft[0:900])

In [ ]:
plt.plot(apple_M.finalday)
plt.plot(data_apple[0:900])

**LCS**

In [ ]:
class StockProblem(Scenario):

    def __init__(self, p, t, input_size=10):
        training_cycles=len(p)-1
        self.p=p
        self.t=t
        self.input_size = input_size
        self.possible_actions = ('Buy', 'Sell', 'Pass')
        self.initial_training_cycles = training_cycles
        self.remaining_cycles = training_cycles


    @property
    def is_dynamic(self):
        return False
        
    def get_possible_actions(self):
        return self.possible_actions
    
    def reset(self):
        self.remaining_cycles = self.initial_training_cycles

        
    def more(self):
        return self.remaining_cycles > 0
    
    def sense(self):
        haystack = BitString(self.p[self.remaining_cycles])
        return haystack
    
    def execute(self, action):
        self.remaining_cycles -= 1
        return action == self.t[self.remaining_cycles]

In [ ]:
class XCS:
        
    def __init__(self,id):
        self.id=id
        self.gain=0 #guadagno
        self.cash=0 
        self.stock=0
        
    def PriceBuy(self, priceS):
        return priceS+(float(priceS)*0.001)
   
    def PriceSell(self, priceB):
        return priceB-(float(priceB)*0.001)
    
    def CreateEnv(self,data, vol):
        self.environment=[]
        for i in range(10, len(data)-5):
            l=[]
            #prima regola
            if(data[i]>=np.mean(data[i-5:i])):
                l.append('1')
            else:
                l.append('0')

            #seconda regola
            if(data[i]>=np.mean(data[i-10:i])):
                l.append('1')
            else:
                l.append('0') 

            #terza regola
            if(data[i]/data[i-2]>1.02):
                l.append('1')
            else:
                l.append('0')

            #quarta regola
            if(data[i]/data[i-2]<(1/1.02)):
                l.append('1')
            else:
                l.append('0')

            #quinta regola
            if(np.mean(data[i-10:i])-np.mean(data[i-5:i])>0):
                l.append('1')
            else:
                l.append('0')

            #sesta regola
            if(np.mean(data[i-5:i])-np.mean(data[i-10:i-5])<0):
                l.append('1')
            else:
                l.append('0')

            #settima regola
            if(data[i]-data[i-10]>0):
                l.append('1')
            else:
                l.append('0')

            #ottava regola
            if(data[i]-data[i-5]>0):
                l.append('1')
            else:
                l.append('0')

            #REGOLE SUI VOLUMI
            devvol=np.std(vol)
            volmedio=np.mean(vol)
            if (vol[i]==0):
                vol[i]=volmedio
            if((vol[i]-volmedio) > devvol):
                l.append('1')
            else:
                l.append('0')

            if((volmedio-vol[i]) > devvol):
                l.append('1')
            else:
                l.append('0')

            #AZIONE
            if(data[i+5]/data[i]>1.015):
                l.append('Buy') 
            elif(data[i+5]/data[i]<1/1.015):
                l.append('Sell')
            else:
                l.append('Pass')

            self.environment.append(l)
            
        return self.environment
    
    def formattation(self, environment):
        self.p=[]
        f='' 
        for i in range (len(self.environment)):
            f=''
            for j in range (len(self.environment[1])-1):
               f+=self.environment[i][j]
            self.p.append(f)
            
        self.t=[] 
        for i in range (len(self.environment)):
            self.t.append(self.environment[i][len(self.environment[0])-1])
    
    
    def train (self):
        problem= StockProblem(self.p,self.t)
        scenario = ScenarioObserver(problem)
        algorithm = XCSAlgorithm()
        
        #xcs' parameters settings
        algorithm.exploration_probability = .001
        algorithm.ga_threshold = 100 
        algorithm.crossover_probability = .4 
        algorithm.do_action_set_subsumption = False
        algorithm.do_ga_subsumption = False
        algorithm.wildcard_probability = .1
        algorithm.deletion_threshold = 0 
        algorithm.mutation_probability = .0001
        
        model = algorithm.new_model(scenario)
        model.run(scenario, learn=True)
        
        self.model=model
        
    def Action(self,statomerc):
        countS=0
        countB=0
        countP=0
        for rule in self.model:
            if (rule.fitness>0.5 and rule.experience>15):
                perf=0
                canc=0
                for j in range (9):

                    if (statomerc[j]==str(rule.condition[j])):
                        perf+=1
                    if (rule.condition[j]==None):
                        canc+=1    
                    if ((canc+perf)==9 and canc<8):
                        if(rule.action=='Buy'):
                            countB+=rule.fitness*rule.numerosity

                        elif (rule.action=='Sell'):
                            countS+=rule.fitness*rule.numerosity
                        else:
                            countP+=rule.fitness*rule.numerosity
        if (countB==0 and countS==0 and countP==0):
            countP=1

        if (countB>countS and countB>countP):
            return('Buy')
        elif (countS>countB and countS>countP):
            return('Sell')
        else:
            return('Pass')
            
            
    def marketstate(self, close, closevol):             
        l=[]
        #I rule
        if(close[-1]>=np.mean(close[-5:])):
            l.append('1')
        else:
            l.append('0')

        #II rule
        if(close[-1]>=np.mean(close[-10:])):
            l.append('1')
        else:
            l.append('0') 

        #III rule
        if(close[0]/close[-2]>1.02):
            l.append('1')
        else:
            l.append('0')

        #IV rule
        if(close[0]/close[-2]<(1/1.02)):
            l.append('1')
        else:
            l.append('0')

        #V rule
        if(np.mean(close[-10:])-np.mean(close[-5:])>0):
            l.append('1')
        else:
            l.append('0')

        #VII rule
        if(close[0]-close[-10]>0):
            l.append('1')
        else:
            l.append('0')

        #VIII rule
        if(close[0]-close[-5]>0):
            l.append('1')
        else:
            l.append('0')

        #vuolumes' rules
        devvol=np.std(closevol)
        volmedio=np.mean(closevol)
        if((closevol[0]-volmedio) > devvol):
            l.append('1')
        else:
            l.append('0')

        if((volmedio-closevol[0]) > devvol):
            l.append('1')
        else:
            l.append('0')

        return (l)
        

In [ ]:
class IndexProblem(Scenario):

    def __init__(self, p, t, input_size=10):
        training_cycles=len(p)-1
        self.p=p
        self.t=t
        self.input_size = input_size
        self.possible_actions = ('Buy', 'Sell', 'Pass')
        self.initial_training_cycles = training_cycles
        self.remaining_cycles = training_cycles


    @property
    def is_dynamic(self):
        return False
        
    def get_possible_actions(self):
        return self.possible_actions
    
    def reset(self):
        self.remaining_cycles = self.initial_training_cycles

        
    def more(self):
        return self.remaining_cycles > 0
    
    def sense(self):
        haystack = BitString(self.p[self.remaining_cycles])
        return haystack
    
    def execute(self, action):
        self.remaining_cycles -= 1
        return action == self.t[self.remaining_cycles]

In [ ]:
class XCS_index:
        
    def __init__(self,id):
        self.id=id
        self.gain=0 #guadagno
        self.cash=0 
        self.stock=0
        
    def PriceBuy(self, priceS):
        return priceS+(float(priceS)*0.001)
   
    def PriceSell(self, priceB):
        return priceB-(float(priceB)*0.001)
    
    def CreateEnv(self,data, data1, data2, data3, data4):
        self.environment=[]
        for i in range(10, len(data)-5):
            l=[]
            
            if (data[i]/data[i-5]>1.015):
                l.append('1')
            else:
                l.append('0')
            
            if (data[i]/data[i-5]<1/1.015):
                l.append('1')
            else:
                l.append('0')
            
            if (np.mean(data[i-10:i])-np.mean(data[i-15:i-5])):
                l.append('1')
            else:
                l.append('0')
            
            if (data[i]>np.mean(data[i-5:i])):
                l.append('1')
            else:
                l.append('0')
            
            if (data[i]>np.mean(data[i-10:i])):
                l.append('1')
            else:
                l.append('0')
            
            if (np.mean(data1[i-5:i])/np.mean(data1[i-10:i-5])>1.015 or np.mean(data2[i-5:i])/np.mean(data2[i-10:i-5])>1.015 or np.mean(data3[i-5:i])/np.mean(data3[i-10:i-5])>1.015 or np.mean(data4[i-5:i])/np.mean(data4[i-10:i-5])>1.015):
                l.append('1')
            else:
                l.append('0')
                
            if (np.mean(data1[i-5:i])/np.mean(data1[i-10:i-5])<(1/1.015) or np.mean(data2[i-5:i])/np.mean(data2[i-10:i-5])<(1/1.015) or np.mean(data3[i-5:i])/np.mean(data3[i-10:i-5])<1/1.015 or np.mean(data4[i-5:i])/np.mean(data4[i-10:i-5])<1/1.015):
                l.append('1')
            else:
                l.append('0')
                
            if((data1[i]-data1[i-5]>0 and data2[i]-data2[i-5]>0 and data3[i]-data3[i-5]>0) or (data1[i]-data1[i-5]>0 and data2[i]-data2[i-5]>0 and data4[i]-data4[i-5]>0) or (data1[i]-data1[i-5]>0 and data3[i]-data3[i-5]>0 and data4[i]-data4[i-5]>0) or (data2[i]-data2[i-5]>0 and data3[i]-data3[i-5]>0 and data4[i]-data4[i-5]>0)):
                l.append('1')
            else:
                l.append('0')
                
            if((data1[i]-data1[i-5]<0 and data2[i]-data2[i-5]<0 and data3[i]-data3[i-5]<0) or (data1[i]-data1[i-5]<0 and data2[i]-data2[i-5]<0 and data4[i]-data4[i-5]<0) or (data1[i]-data1[i-5]<0 and data3[i]-data3[i-5]<0 and data4[i]-data4[i-5]<0) or (data2[i]-data2[i-5]<0 and data3[i]-data3[i-5]<0 and data4[i]-data4[i-5]<0)):
                l.append('1')
            else:
                l.append('0')
                
            if(data[i+5]/data[i]>1.015):
                l.append('Buy') 
            elif(data[i+5]/data[i]<1/1.015):
                l.append('Sell')
            else:
                l.append('Pass')
            
            self.environment.append(l)
        
        return self.environment
            
            
            
            



      
    
    def formattation(self, environment):
        self.p=[]
        f='' 
        for i in range (len(self.environment)):
            f=''
            for j in range (len(self.environment[1])-1):
                f+=self.environment[i][j]
            self.p.append(f)
            
        self.t=[] 
        for i in range (len(self.environment)):
            self.t.append(self.environment[i][len(self.environment[0])-1])
    
    
    def train (self):
        problem= IndexProblem(self.p,self.t)
        scenario = ScenarioObserver(problem)
        algorithm = XCSAlgorithm()
        
        #xcs' parameters settings
        algorithm.exploration_probability = .001
        algorithm.ga_threshold = 100 
        algorithm.crossover_probability = .4 
        algorithm.do_action_set_subsumption = False
        algorithm.do_ga_subsumption = False
        algorithm.wildcard_probability = .1
        algorithm.deletion_threshold = 0 
        algorithm.mutation_probability = .0001
        
        model = algorithm.new_model(scenario)
        model.run(scenario, learn=True)
        
        self.model=model
        
    def Action(self,statomerc):
        countS=0
        countB=0
        countP=0
        for rule in self.model:
            if (rule.fitness>0.5 and rule.experience>15):
                perf=0
                canc=0
                for j in range (9):

                    if (statomerc[j]==str(rule.condition[j])):
                        perf+=1
                    if (rule.condition[j]==None):
                        canc+=1    
                    if ((canc+perf)==9 and canc<8):
                        if(rule.action=='Buy'):
                            countB+=rule.fitness*rule.numerosity

                        elif (rule.action=='Sell'):
                            countS+=rule.fitness*rule.numerosity
                        else:
                            countP+=rule.fitness*rule.numerosity
        if (countB==0 and countS==0 and countP==0):
            countP=1

        if (countB>countS and countB>countP):
            return('Buy')
        elif (countS>countB and countS>countP):
            return('Sell')
        else:
            return('Pass')


    def marketstate(self, data, data1, data2, data3, data4):
        l=[]

        if (data[-1]/data[-6]>1.015):
            l.append('1')
        else:
            l.append('0')

        if (data[-1]/data[-6]<1/1.015):
            l.append('1')
        else:
            l.append('0')

        if (np.mean(data[-11:])-np.mean(data[-16:-5])):
            l.append('1')
        else:
            l.append('0')

        if (data[-1]>np.mean(data[-6:])):
            l.append('1')
        else:
            l.append('0')

        if (data[-1]>np.mean(data[-11:])):
            l.append('1')
        else:
            l.append('0')

        if (np.mean(data1[-6:])/np.mean(data1[-11:-5])>1.015 or np.mean(data2[-6:])/np.mean(data2[-11:-5])>1.015 or np.mean(data3[-6:])/np.mean(data3[-11:-5])>1.015 or np.mean(data4[-6:])/np.mean(data4[-11:-5])>1.015):
            l.append('1')
        else:
            l.append('0')

        if (np.mean(data1[-6:])/np.mean(data1[-11:-5])<1.015 or np.mean(data2[-6:])/np.mean(data2[-11:-5])<1.015 or np.mean(data3[-6:])/np.mean(data3[-11:-5])<1.015 or np.mean(data4[-6:])/np.mean(data4[-11:-5])<1.015):
            l.append('1')
        else:
            l.append('0')

        if((data1[-1]-data1[-6]>0 and data2[-1]-data2[-6]>0 and data3[-1]-data3[-6]>0) or (data1[-1]-data1[-6]>0 and data2[-1]-data2[-6]>0 and data4[-1]-data4[-6]>0) or (data1[-1]-data1[-6]>0 and data3[-1]-data3[-6]>0 and data4[-1]-data4[-6]>0) or (data2[-1]-data2[-6]>0 and data3[-1]-data3[-6]>0 and data4[-1]-data4[-6]>0)):
            l.append('1')
        else:
            l.append('0')

        if((data1[-1]-data1[-6]<0 and data2[-1]-data2[-6]<0 and data3[-1]-data3[-6]<0) or (data1[-1]-data1[-6]<0 and data2[-1]-data2[-6]<0 and data4[-1]-data4[-6]<0) or (data1[-1]-data1[-6]<0 and data3[-1]-data3[-6]<0 and data4[-1]-data4[-6]<0) or (data2[-1]-data2[-6]<0 and data3[-1]-data3[-6]<0 and data4[-1]-data4[-6]<0)):
            l.append('1')
        else:
            l.append('0')

        return (l)


In [ ]:
A=XCS(len(agentlist))
A.CreateEnv(data_microsoft,vol_microsoft)
A.formattation(A.environment)
A.train()
B=XCS(len(agentlist)+1)
B.CreateEnv(data_apple,vol_apple)
B.formattation(B.environment)
B.train()
C=XCS(len(agentlist)+2)
C.CreateEnv(data_amazon,vol_amazon)
C.formattation(C.environment)
C.train()
D=XCS(len(agentlist)+3)
D.CreateEnv(data_toyota,vol_toyota)
D.formattation(D.environment)
D.train()
E=XCS_index(len(agentlist)+4)
E.CreateEnv(index_M.finalday, data_amazon, data_apple, data_microsoft, data_toyota)
E.formattation(E.environment)
E.train()



In [ ]:
agentlist.append(A)
agentlist.append(B)
agentlist.append(C)
agentlist.append(D)
agentlist.append(E)

In [ ]:
a=0
b=0
for t in range(900, 1300):
    for i in range (2000):

        
        if i<250:
            a=0.2
            b=rd.random()
        elif (i<1796):
            a=rd.random()
            b=rd.random()
        else:
            a=0.95
            b=rd.random()*0.85
            
            
        if (b>0.8):
            market=list_M[4]
            if a>0.608:
                agent=agentlist[2]
            elif a<0.6:
                agent=agentlist[0]
            else:
                agent=agentlist[7]

        elif (b>0.6):
            market=list_M[3]
            if (a>0.758):
                agent=agentlist[1]
            elif(a<0.75):
                agent=agentlist[0]
            else:
                agent=agentlist[3]

        elif (b>0.4):
            market=list_M[2]
            if (a>0.758):
                agent=agentlist[1]
            elif (a<0.75):
                agent=agentlist[0]
            else:
                agent=agentlist[5]

        elif (b>0.2):
            market=list_M[1]
            if (a>0.758):
                agent=agentlist[1]
            elif (a<0.75):
                agent=agentlist[0]
            else:
                agent=agentlist[4]
                
        else:
            market=list_M[0]
            if (a>0.758):
                agent=agentlist[1]
            elif (a<0.75):
                agent=agentlist[0]
            else:
                agent=agentlist[6]
        
        if(type(agent)==RandomAgent):
            action=agent.Action()
            if (action=='Buy'):
                if (type(market)==Market):
                    market.modifyqueueB(agent.Price(market.data[t-1]),agent.id)
                else:
                    market.modifyqueueB(agent.Price(market.finalday[t-1]),agent.id)
                if (market.transaction(agentlist)==True and type(market)==Market):
                    index_M.modify_currentsubprice()
            if (action=='Sell'):
                if (type(market)==Market):
                    market.modifyqueueS(agent.Price(market.data[t-1]),agent.id)
                else:
                    market.modifyqueueS(agent.Price(market.finalday[t-1]),agent.id)
                if (market.transaction(agentlist)==True and type(market)==Market):
                    index_M.modify_currentsubprice()


        elif(type(agent)==Arbitrajeur and type(market)!=Market_Index):
            action=agent.Action(market.data[t], market.currentprice)
            if (action=='Buy'):
                if (len(market.queueS)!=0):
                    market.modifyqueueB(agent.PriceBuy(market.queueS[0][0]),agent.id)
                else:
                    market.modifyqueueB(agent.PriceBuy(market.currentprice*1.01),agent.id)
                if (market.transaction(agentlist)==True and type(market)==Market):
                    index_M.modify_currentsubprice()
            if (action=='Sell'):
                if (len(market.queueB)!=0):
                    market.modifyqueueS(agent.PriceSell(market.queueB[-1][0]),agent.id)
                else:
                    market.modifyqueueB(agent.PriceBuy(market.currentprice*(1/1.01)),agent.id)
                if (market.transaction(agentlist)==True and type(market)==Market):
                    index_M.modify_currentsubprice()


        elif(type(agent)==Arbitrajeur_index and type(market)!=Market):
            action=agent.Action(market.subprice, market.currentprice)
            if (action=='Buy'):
                if (len(market.queueS)!=0):
                    market.modifyqueueB(agent.PriceBuy(market.queueS[0][0]),agent.id)
                else:
                    market.modifyqueueB(agent.PriceBuy(market.currentprice*1.01),agent.id)
                market.transaction(agentlist)
                
                for i in range (0,3):
                    if (len(list_M[i].queueB)!=0):
                        list_M[i].modifyqueueS(agent.PriceSell(list_M[i].queueB[-1][0]),agent.id)
                    else:
                        list_M[i].modifyqueueB(agent.PriceBuy(list_M[i].currentprice*(1/1.01)),agent.id)
                    if (list_M[i].transaction(agentlist)==True and type(list_M[i])==Market):
                        index_M.modify_currentsubprice()
                    
                    
            if (action=='Sell'):
                if (len(market.queueB)!=0):
                    market.modifyqueueS(agent.PriceSell(market.queueB[-1][0]),agent.id)
                else:
                    market.modifyqueueB(agent.PriceBuy(market.currentprice*(1/1.01)),agent.id)
                market.transaction(agentlist)
            
                for i in range (0,3):
                    if (len(list_M[i].queueS)!=0):
                        list_M[i].modifyqueueB(agent.PriceBuy(list_M[i].queueS[0][0]),agent.id)
                    else:
                        list_M[i].modifyqueueB(agent.PriceBuy(list_M[i].currentprice*1.01),agent.id)
                    market.transaction(agentlist)
                    

        elif(type(agent)==XCS and type(market)!=Market_Index):
            action=agent.Action(agent.marketstate(market.finalday, market.volume[:t]))
            if (action=='Buy'):
                if (len(market.queueS)!=0):
                    market.modifyqueueB(agent.PriceBuy(market.queueS[0][0]),agent.id)
                else:
                    market.modifyqueueB(agent.PriceBuy(market.currentprice*1.01),agent.id)
                if (market.transaction(agentlist)==True and type(market)==Market):
                    index_M.modify_currentsubprice()
            if (action=='Sell'):
                if (len(market.queueB)!=0):
                    market.modifyqueueS(agent.PriceSell(market.queueB[-1][0]),agent.id)
                else:
                    market.modifyqueueB(agent.PriceBuy(market.currentprice*(1/1.01)),agent.id)
                if (market.transaction(agentlist)==True and type(market)==Market):
                    index_M.modify_currentsubprice()    
                    

        elif(type(agent)==XCS_index and type(market)==Market_Index):
            action=agent.Action(agent.marketstate(market.finalday, microsoft_M.finalday,toyota_M.finalday, apple_M.finalday, amazon_M.finalday))
            if (action=='Buy'):
                if (len(market.queueS)!=0):
                    market.modifyqueueB(agent.PriceBuy(market.queueS[0][0]),agent.id)
                else:
                    market.modifyqueueB(agent.PriceBuy(market.currentprice*1.01),agent.id)
            if (action=='Sell'):
                if (len(market.queueB)!=0):
                    market.modifyqueueS(agent.PriceSell(market.queueB[-1][0]),agent.id)
                else:
                    market.modifyqueueB(agent.PriceBuy(market.currentprice*(1/1.01)),agent.id)


                    
    for market in list_M:
        market.finalday.append(market.currentprice)
        if type(market)==Market_Index:
            market.finaldaysub.append(market.subprice)
        market.deletequeue

In [ ]:
plt.plot(index_M.finalday)
plt.plot(index_M.finaldaysub)


In [ ]:
plt.plot(apple_M.finalday)
plt.plot(data_apple[0:1300])

In [ ]:
plt.plot(toyota_M.finalday)
plt.plot(data_toyota[0:1300])

In [ ]:
plt.plot(amazon_M.finalday)
plt.plot(data_amazon[0:1300])

In [ ]:
plt.plot(microsoft_M.finalday)
plt.plot(data_microsoft[0:1300])

In [ ]:
A.gain=A.stock*microsoft_M.currentprice+A.cash

In [ ]:
A.gain

In [ ]:
B.gain=B.stock*apple_M.currentprice+B.cash

In [ ]:
B.cash

In [ ]:
C.stock

In [ ]:
D.stock

In [ ]:
E.gain=E.stock*microsoft_M.currentprice+E.cash

In [ ]:
E.gain

In [ ]:
for rule in B.model:
    print(rule.condition[0])

In [ ]:
a=['0', '1']
b=[0,1]
for i in range(0,2):
    if (a[i]==str(b[i])):
        print('ciao')